(sec:T6:ejemplo_aliasing)=
### Muestreo

Un ejemplo de muestreo con aliasing para el tema 6  

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath(".."))

from utils.plot_helpers import style_math_axes

In [2]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Label, Span
from bokeh.io import output_notebook

# Importamos tus helpers
from utils.plot_helpers import style_math_axes, add_math_ticks

output_notebook()

# ==========================================
# 1. PARÁMETROS INICIALES
# ==========================================
# Escenario
T_MAX = 1.0        # 1 segundo de visualización
RES = 1000         # Resolución para dibujar curvas continuas

# Estado inicial
f0_init = 5.0      # Señal de 5 Hz
fs_init = 20.0     # Muestreo a 20 Hz (Sobremuestreo, todo bien)

# Generación de vectores de tiempo para dibujo continuo
t_cont = np.linspace(0, T_MAX, RES)

# ==========================================
# 2. LÓGICA DEL ALIASING (PYTHON)
# ==========================================
def get_alias_freq(f_sig, f_samp):
    """
    Calcula la frecuencia aparente (alias) más baja en banda base [-fs/2, fs/2].
    """
    # Ratio f / fs
    ratio = f_sig / f_samp
    # Restamos el entero más cercano (rounding)
    # Ejemplo: si ratio es 0.9 (fs=10, f=9), round(0.9)=1 -> alias = -0.1 -> -1Hz
    ratio_alias = ratio - np.round(ratio)
    return ratio_alias * f_samp

# Datos iniciales
y_analog = np.cos(2 * np.pi * f0_init * t_cont)

f_alias = get_alias_freq(f0_init, fs_init)
y_rec = np.cos(2 * np.pi * f_alias * t_cont) # Señal reconstruida (la "mentira")

# Puntos de muestreo
# Generamos indices n hasta cubrir T_MAX
n_samples = int(T_MAX * fs_init) + 1
t_n = np.arange(n_samples) / fs_init
y_n = np.cos(2 * np.pi * f0_init * t_n)

# ==========================================
# 3. FUENTES DE DATOS
# ==========================================

# Señales continuas (Analogica vs Reconstruida)
source_waves = ColumnDataSource(data=dict(
    t=t_cont,
    y_ana=y_analog,  # La verdad (Gris)
    y_rec=y_rec      # La reconstrucción (Azul)
))

# Muestras (Puntos rojos)
source_samples = ColumnDataSource(data=dict(
    t=t_n,
    y=y_n
))

# Etiquetas informativas
source_info = ColumnDataSource(data=dict(
    text_f0=[f"f₀ = {f0_init:.1f} Hz"],
    text_fs=[f"fs = {fs_init:.1f} Hz"],
    text_alias=[f"f_aparente = {abs(f_alias):.1f} Hz"]
))


# ==========================================
# 4. GRÁFICO
# ==========================================
p = figure(width=700, height=350, title="Efecto Aliasing: ¿Ves la señal real o el fantasma?")
style_math_axes(p, x_range=(-0.05, T_MAX + 0.1), y_range=(-1.5, 1.8), xlabel="t (s)", ylabel="Amplitud")

# 1. Señal Analógica (La Realidad) - Fantasma Gris
p.line('t', 'y_ana', source=source_waves, color="gray", alpha=0.4, line_width=2, line_dash="dashed", legend_label="Señal Real (Analógica)")

# 2. Señal Reconstruida (Lo que vemos) - Azul
p.line('t', 'y_rec', source=source_waves, color="#1f77b4", line_width=3, alpha=0.8, legend_label="Señal Reconstruida (Alias)")

# 3. Muestras (Los "Stems")
# Líneas verticales
p.segment(x0='t', y0=0, x1='t', y1='y', source=source_samples, color="#d62728", line_width=2)
# Puntos
p.scatter('t', 'y', source=source_samples, color="#d62728", size=8, legend_label="Muestras")

p.legend.location = "top_right"
p.legend.background_fill_alpha = 0.9

# Etiquetas de estado
lbl_info = Label(x=0, y=1.6, text_font_size="11pt", text_color="#333333",
                 text="Nyquist se cumple: fs > 2·f₀") # Texto inicial estático, luego lo controla JS
p.add_layout(lbl_info)


# ==========================================
# 5. INTERACCIÓN (JS)
# ==========================================

s_f0 = Slider(start=1, end=20, value=f0_init, step=0.5, title="Frecuencia Señal (f₀)")
s_fs = Slider(start=1, end=40, value=fs_init, step=0.5, title="Frecuencia Muestreo (fs)")

callback = CustomJS(
    args=dict(source_w=source_waves, 
              source_s=source_samples, 
              lbl=lbl_info,
              s_f0=s_f0, s_fs=s_fs, 
              T_MAX=T_MAX, RES=RES),
    code="""
    const f0 = s_f0.value;
    const fs = s_fs.value;
    const two_pi = 2 * Math.PI;
    
    // --- 1. CALCULO DEL ALIAS ---
    // Formula: ratio = f0/fs; alias = (ratio - round(ratio)) * fs
    const ratio = f0 / fs;
    const ratio_alias = ratio - Math.round(ratio);
    const f_alias = ratio_alias * fs;
    
    // Si f_alias es negativo, significa cambio de fase (180 deg)
    // Visualmente es lo mismo que cos(-x) = cos(x), pero 
    // conceptualmente es una frecuencia negativa.
    
    // Actualizar etiqueta
    const nyquist = fs / 2;
    let status_text = "";
    if (f0 < nyquist) {
        status_text = "OK (fs > 2f₀). Frecuencia detectada: " + Math.abs(f_alias).toFixed(2) + " Hz";
        lbl.text_color = "#2ca02c"; // Verde
    } else {
        status_text = "⚠️ ALIASING! (fs < 2f₀). Parece de: " + Math.abs(f_alias).toFixed(2) + " Hz";
        lbl.text_color = "#d62728"; // Rojo
    }
    lbl.text = status_text;
    
    
    // --- 2. ONDAS CONTINUAS ---
    const t = source_w.data['t'];
    const y_ana = source_w.data['y_ana'];
    const y_rec = source_w.data['y_rec'];
    
    for (let i = 0; i < t.length; i++) {
        // Señal real
        y_ana[i] = Math.cos(two_pi * f0 * t[i]);
        // Señal percibida (usa la frecuencia alias)
        y_rec[i] = Math.cos(two_pi * f_alias * t[i]);
    }
    source_w.change.emit();
    
    
    // --- 3. MUESTRAS (Re-generar array de tiempos) ---
    // En JS rellenamos los arrays de muestras dinámicamente
    // Número de muestras = T_MAX * fs
    const n_samples = Math.floor(T_MAX * fs) + 1;
    
    // Necesitamos redimensionar los arrays del datasource si cambia el tamaño
    // La forma más segura en BokehJS es crear nuevos arrays y asignarlos
    const new_t_n = new Float64Array(n_samples);
    const new_y_n = new Float64Array(n_samples);
    
    for (let n = 0; n < n_samples; n++) {
        let time = n / fs;
        new_t_n[n] = time;
        new_y_n[n] = Math.cos(two_pi * f0 * time);
    }
    
    source_s.data['t'] = new_t_n;
    source_s.data['y'] = new_y_n;
    source_s.change.emit();
""")

s_f0.js_on_change('value', callback)
s_fs.js_on_change('value', callback)

layout = column(
    row(s_f0, s_fs),
    p,
    sizing_mode="scale_width"
)

show(layout)

Loading BokehJS ...